In [5]:
for msg in messages:
    msg_detail = service.users().messages().get(userId='me', id=msg['id'], format='full').execute()
    headers = msg_detail['payload']['headers']
    
    for header in headers:
        if header['name'] == 'From':
            sender = header['value']
        if header['name'] == 'Subject':
            subject = header['value']
    
    # Extract email body (if available)
    if 'parts' in msg_detail['payload']:
        body = msg_detail['payload']['parts'][0]['body'].get('data', '')
    else:
        body = "No Body Available"

    print(f"From: {sender}")
    print(f"Subject: {subject}")
    print(f"Body: {body[:100]}...")  # Print first 100 characters of the body
    print("-" * 50)


NameError: name 'messages' is not defined

In [1]:
pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client


In [ ]:
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

flow = InstalledAppFlow.from_client_secrets_file("client_secret_.json", SCOPES)
creds = flow.run_local_server(port=8081)
service = build('gmail', 'v1', credentials=creds)

# Fetch Emails
results = service.users().messages().list(userId='me', maxResults=5).execute()
messages = results.get('messages', [])

for msg in messages:
    msg_detail = service.users().messages().get(userId='me', id=msg['id']).execute()
    print(msg_detail['snippet'])  # Short email preview



In [1]:
import base64
from email.mime.text import MIMEText
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from datetime import datetime, timedelta

# Define Gmail and Calendar API scopes
GMAIL_SCOPES = ['https://www.googleapis.com/auth/gmail.send']
CALENDAR_SCOPES = ['https://www.googleapis.com/auth/calendar']

def authenticate_api(scopes):
    """Authenticate with Google API."""
    flow = InstalledAppFlow.from_client_secrets_file("client_secret_.json", scopes)
    creds = flow.run_local_server(port=8081)
    return build('gmail', 'v1', credentials=creds)

def schedule_meeting_and_send_email(summary, start_time, duration_minutes, attendees):
    """Schedule a meeting and send email invitations."""
    # Calculate end time
    start_datetime = datetime.strptime(start_time, "%Y-%m-%dT%H:%M:%S")
    end_datetime = start_datetime + timedelta(minutes=duration_minutes)
    
    # Format dates for calendar event
    start_str = start_datetime.strftime("%Y-%m-%dT%H:%M:%S")
    end_str = end_datetime.strftime("%Y-%m-%dT%H:%M:%S")

    # Meeting details
    meeting_link = f"https://meet.google.com/{start_datetime.strftime('%H%M%S')}"  # Dummy link
    email_body = f"""
    📅 **Meeting Scheduled!**

    📝 **Topic:** {summary}
    🗓️ **Date:** {start_datetime.strftime('%A, %d %B %Y')}
    ⏰ **Time:** {start_datetime.strftime('%I:%M %p')}
    ⏳ **Duration:** {duration_minutes} minutes
    🔗 **Join Here:** {meeting_link}

    📍 Location: Virtual (Google Meet)
    """

    # Authenticate Gmail API
    gmail_service = authenticate_api(GMAIL_SCOPES)

    # Send email to each participant
    for recipient in attendees:
        message = MIMEText(email_body, 'plain')
        message['to'] = recipient
        message['subject'] = f"📅 Meeting Scheduled: {summary}"
        raw_message = base64.urlsafe_b64encode(message.as_bytes()).decode()

        # Send email via Gmail API
        gmail_service.users().messages().send(userId="me", body={"raw": raw_message}).execute()
        print(f"✅ Email sent to {recipient}")

# Example usage
attendees = ["purvesh.varale24@sanjivani.edu.in", "varad.ahire24@sanjivani.edu.in"]
schedule_meeting_and_send_email("game time", "2025-02-21T15:50:20", 60, attendees)


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=424078835405-5v64ao8qamfnbemi93u8e67r0i0g5n2t.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8081%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.send&state=5qj0gWHGNO4PU8VMa9iER4zP9Ix1u2&access_type=offline
✅ Email sent to purvesh.varale24@sanjivani.edu.in
✅ Email sent to varad.ahire24@sanjivani.edu.in


In [1]:
# to add claender in the application 

In [7]:
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from datetime import datetime, timedelta

# Define Calendar API scope
SCOPES = ['https://www.googleapis.com/auth/calendar']

def authenticate_calendar():
    """Authenticate and return Google Calendar API service."""
    flow = InstalledAppFlow.from_client_secrets_file("client_secret_.json", SCOPES)
    creds = flow.run_local_server(port=8081)
    return build('calendar', 'v3', credentials=creds)

def create_calendar_event(summary, start_time, duration_minutes, attendees):
    """Create a Google Calendar event and send invites."""
    service = authenticate_calendar()

    # Calculate start & end times
    start_datetime = datetime.strptime(start_time, "%Y-%m-%dT%H:%M:%S")
    end_datetime = start_datetime + timedelta(minutes=duration_minutes)

    # Format times in ISO 8601 format
    event = {
        'summary': summary,
        'start': {'dateTime': start_datetime.isoformat(), 'timeZone': 'Asia/Kolkata'},
        'end': {'dateTime': end_datetime.isoformat(), 'timeZone': 'Asia/Kolkata'},
        'attendees': [{'email': email} for email in attendees],
        'reminders': {'useDefault': True},
    }

    event = service.events().insert(calendarId='primary', body=event).execute()
    print(f"✅ Meeting scheduled: {event['htmlLink']}")

# Example usage
attendees = ["john@example.com", "sarah@example.com"]
create_calendar_event("Project Sync", "2025-02-18T15:00:00", 60, attendees)


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=424078835405-5v64ao8qamfnbemi93u8e67r0i0g5n2t.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8081%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar&state=0QCY53PgEVZR8AIZn8GpSNxSelTeTg&access_type=offline


HttpError: <HttpError 403 when requesting https://www.googleapis.com/calendar/v3/calendars/primary/events?alt=json returned "Google Calendar API has not been used in project 424078835405 before or it is disabled. Enable it by visiting https://console.developers.google.com/apis/api/calendar-json.googleapis.com/overview?project=424078835405 then retry. If you enabled this API recently, wait a few minutes for the action to propagate to our systems and retry.". Details: "[{'message': 'Google Calendar API has not been used in project 424078835405 before or it is disabled. Enable it by visiting https://console.developers.google.com/apis/api/calendar-json.googleapis.com/overview?project=424078835405 then retry. If you enabled this API recently, wait a few minutes for the action to propagate to our systems and retry.', 'domain': 'usageLimits', 'reason': 'accessNotConfigured', 'extendedHelp': 'https://console.developers.google.com'}]">